In [17]:
import numpy as np
import pandas as pd
import math

import sys
import types

import RPI
import NBS
import RSD
import RPIk

np.set_printoptions(precision=3)

In [18]:
# Returns size x size matrices that, in theory, should yield an RSD:NBS worst case approximation of size 
# Takes the size of matrix (# of agents and items)
def n_approx_val_generator(size):
    
    eps = sys.float_info.epsilon
    V = np.zeros(shape=(size, size))
    
    V[0][0] = 1
    for i in range(1, size):
        V[i][0] = 1
        V[i][i] = float(1-eps)
    
    return V

In [19]:
def i_mat(size):
    V = np.zeros((size, size))
    for i in range(size):
        V[i][i] = 1
    return V

In [20]:
def monte_carlo_RPI(V, n):
    max_size = max(V.shape[0], V.shape[1])
    p = np.zeros((max_size, max_size))
    for i in range(n):
        p = np.add(p, RPI.RPI(V, 4))
        
    ax_0 = []
    for i in range(V.shape[0], p.shape[0]):
        ax_0.append(i)
    p = np.delete(p, ax_0, 0)
    
    ax_1 = []
    for j in range(V.shape[1], p.shape[1]):
        ax_1.append(j)
    p = np.delete(p, ax_1, 1)
        
    return p/n

In [21]:
def monte_carlo_RPIk(V, n):
    max_size = max(V.shape[0], V.shape[1])
    p = np.zeros((max_size, max_size))
    for i in range(n):
        p = np.add(p, RPIk.RPIk(V, 1))
        
    ax_0 = []
    for i in range(V.shape[0], p.shape[0]):
        ax_0.append(i)
    p = np.delete(p, ax_0, 0)
    
    ax_1 = []
    for j in range(V.shape[1], p.shape[1]):
        ax_1.append(j)
    p = np.delete(p, ax_1, 1)
        
    return p/n

In [22]:
# Returns expected value of agent/item utility matrix determined empirically
# Takes value matrix, optionally number of iterations (default 1000)
def monte_carlo_RSD(V, n):     
    p = np.zeros(V.shape)
    for i in range(n):
        p = np.add(p, RSD.RSD(V))
    return p/n

In [23]:
# Takes num agents, num items, distribution for agent values to be drawn from, and optionally number of trials
# Returns worst-case ratio 
def worst_case_ratio(V, Func, n=1000):
    if not(type(V) == np.ndarray):
        raise Exception("V must be a numpy.ndarray, currently type: ", type(V))
    if not isinstance(Func, types.FunctionType):
        raise Exception("Func must be a function, currently type: ", type(Func))
    if np.ndim(V) != 2:
        raise Exception("V must be a 2-D np.ndarray. Current shape: ", vals.shape)
        
    O = np.zeros(V.shape[0])
    
    P_NBS = NBS.NBS(V,O, np.ones(V.shape[1]))
    if P_NBS is None: 
        raise Exception("NBS returned None")
        
    P_Func = Func(V, n)
        
    U_NBS = np.sum(np.multiply(V, P_NBS), axis=1)
    U_Func = np.sum(np.multiply(V, P_Func), axis=1)
    
    U_ratio = np.divide(U_NBS, U_Func)
    return np.amax(U_ratio)

In [24]:
def same_pref(i, j):
    v = np.zeros(shape=(i,j))
    v[0] = np.random.rand(1, j)
    for k in range(1, i):
        v[k] = v[k-1]
    return v

In [ ]:
for i in range(5, 1000):
    v = i_mat(i)
    v[1][1] = 0
    v[1][0] = 1
    nbs = RPI.RPI(v, 4)
    print("\n-------",i,"--------\n")

In [ ]:
print("V: \n", v)
p = RSD.RSD(v)
print("RSD: \n", p)

In [ ]:
max_size = 11 #exclusive
min_size = 5 #inclusive
assert(min_size >= 4)
assert(max_size > min_size)

wcr_RPIk = np.zeros(shape=(max_size-min_size, max_size-min_size))
wcr_RPI = np.zeros(shape=(max_size-min_size, max_size-min_size))
wcr_RSD = np.zeros(shape=(max_size-min_size, max_size-min_size))

# i is num agents, j is num courses
for i in range(min_size, max_size):
    for j in range(min_size, max_size):
        v = RPI.pref_att(i, j, 0.5)
        wcr_RPIk[i-min_size][j-min_size] = worst_case_ratio(v, monte_carlo_RPIk)
        wcr_RPI[i-min_size][j-min_size] = worst_case_ratio(v, monte_carlo_RPI)
        wcr_RSD[i-min_size][j-min_size] = worst_case_ratio(v, monte_carlo_RSD)
        
        

In [ ]:
print(wcr_RPI)

In [ ]:
print(wcr_RSD)

In [ ]:
print(wcr_RPIk)

In [ ]:
min_size = 5
max_size = 25
wcr_RPIk = np.zeros(max_size-min_size)
wcr_RPI = np.zeros(max_size-min_size)
wcr_RSD = np.zeros(max_size-min_size)

# i is num agents, j is num courses
for i in range(min_size, max_size):
        v = same_pref(i, i)
        
        rpi = worst_case_ratio(v, monte_carlo_RPI)
        print("WCR RPI: \n", rpi)
        wcr_RPI[i-min_size] = rpi
        
        rpik = worst_case_ratio(v, monte_carlo_RPIk)
        print("WCR RPIk: \n", rpik)
        wcr_RPIk[i-min_size] = rpik
        
        rsd = worst_case_ratio(v, monte_carlo_RSD)
        print("RSD: \n", rsd)
        wcr_RSD[i-min_size] = rsd
        
        print("\n---------------------------\n")
        

In [ ]:
print(wcr_RPIk)
print(wcr_RSD)

In [25]:
df = pd.read_csv('/Users/brandosocarras/Documents/Mechanism_Design_20/Course Preferences_August 27, 2020_15.42.csv')
df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q56,Q56_3_TEXT,Q57,Q57_3_TEXT,Q58,Q58_3_TEXT,Q59,Q59_3_TEXT,Q60,Q60_3_TEXT
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,CS 397/497 - Riesbeck - Selected Choice,CS 397/497 - Riesbeck - Want to Enroll - Text,CS 409 - Selected Choice,CS 409 - Want to Enroll - Text,CS 496 - Makarychev - Selected Choice,CS 496 - Makarychev - Want to Enroll - Text,CS 497 - Zhang - Selected Choice,CS 497 - Zhang - Want to Enroll - Text,CS 497 - O'Rourke - Selected Choice,CS 497 - O'Rourke - Want to Enroll - Text
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID56""}","{""ImportId"":""QID56_3_TEXT""}","{""ImportId"":""QID57""}","{""ImportId"":""QID57_3_TEXT""}","{""ImportId"":""QID58""}","{""ImportId"":""QID58_3_TEXT""}","{""ImportId"":""QID59""}","{""ImportId"":""QID59_3_TEXT""}","{""ImportId"":""QID60""}","{""ImportId"":""QID60_3_TEXT""}"
2,2020-08-12 17:26:39,2020-08-12 17:26:50,Survey Preview,NaN,100,11,True,2020-08-12 17:26:51,R_063torU01YuaDv3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-08-13 15:49:58,2020-08-13 15:50:02,Survey Preview,NaN,100,3,True,2020-08-13 15:50:03,R_1BPphpAxrj5kFRR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-08-13 15:52:03,2020-08-13 15:52:08,Survey Preview,NaN,100,5,True,2020-08-13 15:52:09,R_3NOp0xJmLbxJfsP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df.index

RangeIndex(start=0, stop=108, step=1)

In [27]:
df_0 = df[df['Status'] == 'IP Address']

In [28]:
# df_0['Q'+str(24)][14] == 'Not interested'
df_0.index

Int64Index([ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,
             23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
             36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
             49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
             62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
             75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
             88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
            101, 102, 103, 104, 105, 106, 107],
           dtype='int64')

In [29]:
course_vals = np.zeros(shape=(98, 61-24))
delete = []
for j in range(24, 61):
    df_Q = df_0['Q' + str(j)]
    for i in df_0.index:
        if df_Q[i] == 'Want to Enroll':
            ans = df_0['Q'+str(j)+'_3_TEXT'][i]
            try: 
                fl_ans = float(ans)
            except:
                delete.append(i)
                continue
            if np.isnan(fl_ans):
                delete.append(i)
                continue
            course_vals[i-10][j-24] = fl_ans

In [30]:
a = np.sum(course_vals, axis=1)
print(a)
b=[]
for i in range(a.size):
    if a[i] == 0.0:
        if df_0['Q11'][i+10] != 0:
            b.append(i)

[  0.  25.   0.  40. 325.   0.   0. 180.  15.   0.   0.   6.  14.   0.
 350. 300.   0.   5.  46.   0.   4. 120.  30.  41.  14. 410.   1. 580.
 180.   3.   0.   0.   0.   0.  33.  22.  23. 130.   0.   3.   0.  42.
   0. 205.   1.   0.   0.   1. 100.  11.   0.   0. 200.   2. 500.  21.
   3.   0.   5. 325. 200.  21. 380.  12.   9.   0.   9. 225.   0.   0.
   0.   0.   0.   0. 235.  20. 200.   1.   0.   0.   0.   0.  32.  18.
  11.   0.   0.  60. 378. 200.   0.   0. 100.   0.   3.   1. 100. 275.]


In [31]:
len(b)

37

In [32]:
v = np.delete(course_vals, b, axis=0)

In [33]:
def minmax_scale(v):
    f_maxs = np.amax(v, axis=1)
    for i in range(v.shape[0]):
        lmax = f_maxs[i]
        for j in range(v.shape[1]):
            v[i][j] = v[i][j] / lmax
    return v

In [ ]:
# nbs = NBS.NBS(minmax_scale(v), np.zeros(v.shape[0]), np.ones(v.shape[1]))
wcr_RPI = worst_case_ratio(minmax_scale(v), monte_carlo_RPI)
wcr_RPIk = worst_case_ratio(minmax_scale(v), monte_carlo_RPIk)
wcr_RSD = worst_case_ratio(minmax_scale(v), monte_carlo_RSD)

In [ ]:
# 1: Failed on trial 17 index 37
# 2: 
# 
# 1: Passed First Lap, Failed Second Initial PA Allocation. Indices on Failure: 1,4,6,7,11,12,21,22,24,28,39,48,51,57,60
# 2: Failed First Lap, Passed First Initial PA Allocation, Failed on NBS w/o t4 i11. Indices on Failure: 1,2,7,9,11,12,15,17,19,20,22,23,25,26,28,30,32,34,35,36,37,38,40,43,45,50,51,52,54,56,58
# 3: Failed First Lap, Failed First Initial PA Allocation, Indices on Failure: 0,1,5,11,14,15,16,17,19,22,23,24,28,29,32,33,34,37,38,40,43,43,47,48,49,51,52,53,55,57,59
# 4: Passed First Lap, Failed Second Initial PA Allocation. Indices on Failure: 1,2,16,18,19,32,35,37,39,41,48,52,54,56,59
# 5: Passed Second Lap, Passed Third Initial PA Allocation, Failed on NBS w/o t1 i10. Indices on Failure: 10,17,22,26,28,44,46,54
# 6: Passed First Lap, Passed Second Initial PA Alllocation, Failed on NBS w/o t1 i1. Indices on Failure: 1,3,8,13,19,25,28,29,32,37,45,48,57,58,60

In [ ]:
# New Strat
# 
# 

In [ ]:
def pairsq(lsts, next_i):
    